In [1]:
!pip install beautifulsoup4 requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import bs4
from IPython import display
import pandas as pd
from multiprocessing import Pool

In [3]:
def get_page(i=1):
  assert isinstance(i, int) and i >= 1 and i <= 20
  url = f"https://hypeauditor.com/top-youtube/?p={i}"
  resp = requests.get(url)
  soup = bs4.BeautifulSoup(resp.text, "html.parser")

  def select_and_callbacks(
    selector,
    callbacks = [
      lambda elem : elem.encode_contents().decode()
    ]
  ):
    ptr = soup.select(selector)
    if not isinstance(callbacks, list):
      callbacks = [callbacks]
      
    for fun in callbacks:
      ptr = [
        fun(elem) for elem in ptr
      ]
    return ptr
  
  ranks = select_and_callbacks("div.row-cell.rank > span")
  names = select_and_callbacks("div.contributor__name > div")
  followers = select_and_callbacks("div.row-cell.subscribers")
  country = select_and_callbacks("div.row-cell.audience")
  views = select_and_callbacks("div.row-cell.avg-views")
  likes = select_and_callbacks("div.row-cell.avg-likes")

  links = select_and_callbacks(
    "a.contributor-link",
    lambda elem:elem["href"]
  )
  categories = select_and_callbacks(
    "div.row-cell.category",
    lambda subElem: [
      elem.encode_contents().decode()
      for elem in subElem.select("div.tag__content")
    ]
  )

  return pd.DataFrame(
    dict(
      ranks=ranks,
      names=names,
      links=likes,
      categories=categories,
      followers=followers,
      country=country,
      views=views,
      likes=likes
    )
  ).set_index("ranks")

In [4]:
%%time 
# sometime it will have error
with Pool(20) as p:
  table = pd.concat(
    p.map(get_page, range(1,21))
  )

CPU times: user 122 ms, sys: 157 ms, total: 279 ms
Wall time: 5.02 s


In [5]:
table

,names,links,categories,followers,country,views,likes
ranks,,,,,,,
1,tseries,15K,[Music &amp; Dance],225.4M,India,332.7K,15K
2,checkgate,57.6K,[Education],143.6M,,11.1M,57.6K
3,PewDiePie,221.2K,[Animation],111.5M,United States,3.5M,221.2K
4,MrBeast6000,1.9M,"[Video games, Humor]",105.1M,United States,38M,1.9M
5,Like Nastya,21.7K,[Toys],101.4M,,4.7M,21.7K
...,...,...,...,...,...,...,...
996,nhstudioz,46,[],10M,India,6K,46
997,VitalyzdTv,3.4K,[Humor],10M,United States,75.8K,3.4K
998,Nickelodeon,502,"[Movies, Animation]",10M,,60.2K,502


In [6]:
table.to_csv("top1000.csv", sep="\t")